In [4]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import plotly_express as px
import warnings
import collections
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import os
import pymysql
from sqlalchemy import create_engine



##  <font color='CornflowerBlue'><center><strong>I. <ins>The recipe Recommender </ins> </strong><center><font color='blue'></font>


In [8]:
def user_recipe():
    user_recipe = input("Give me a recipe that you like:")
    return user_recipe

def user_final_choice(options_user):
    print(f"Which one of these is the closest to your recipe?")
    for idx, recipe in options_user["recipe"].items():
        print(f"{idx}: {recipe}")
    ind_user = int(input("Type '0', '1', or '2': "))
    return ind_user


#options_user = options_user.reset_index(drop=True)
def user_final_choice(options_user):
    print(f"Which one of these is the closest to your recipe?")
    for idx, recipe in options_user["recipe"].items():
        print(f"{idx}: {recipe}")
    ind_user = int(input("Type '0', '1', or '2': "))
    re_user = options_user.iloc[[ind_user]]
    return re_user 

#re_user = user_final_choice(options_user)

df = pd.read_csv("recipes_10_lem.csv")
df.head(2)

dfs = df[(df["website_id"] != 2)]


#user_recipe = 'steak' 
dfs.reset_index(drop=True, inplace=True)
#dfs.shape


### Taking User input:


In [9]:
def t_user_recipe():
    user_recipe = input("Give me a recipe that you like:")
    return user_recipe

user_recipe = t_user_recipe()

In [12]:
# FONCTION 1 INPUT USER

cv_char = CountVectorizer(analyzer='char_wb', ngram_range = (2,2))

list_recipes = list(dfs["recipe"])
list_recipes.append(user_recipe)

count_matrix = cv_char.fit_transform(list_recipes)
cosine_sim = cosine_similarity(count_matrix)
similar_recipes = list(enumerate(cosine_sim[-1]))
sorted_similar_recipes = sorted(similar_recipes,key=lambda x:x[1],reverse=True)[1:]
suggested_ids = [d[0] for d in sorted_similar_recipes[:3]] # returns just 3 recipes
options_user = dfs.loc[suggested_ids]
options_user = options_user.reset_index(drop=True)


In [13]:
options_user 

,recipe_id,recipe,url,list_ingredients_raw,instructions_raw,category_id,serves,time_prep,nutrients_table,diet_id,...,search_terms,category,meal,meal_id,combined_features,tokens,tokens2,lemmatized,no_stop_word,lems_string
0,106904,lasagna,NaN,"[""2 tablespoons oil"",""1 clove minced ...",NaN,NaN,8,NaN,NaN,3,...,{'dinner'},NaN,NaN,NaN,"lasagna ['oil', 'garlic', 'onion', 'italian sa...","['lasagna', '[', ""'oil"", ""'"", ',', ""'garlic"", ...","['lasagna', 'sausage', 'paste', 'sauce', 'wate...","['lasagna', 'sausage', 'paste', 'sauce', 'wate...","['lasagna', 'sausage', 'paste', 'sauce', 'wate...",lasagna sausage paste sauce water noodle chees...
1,101714,bolognese,NaN,"[""2 (28 ounce) cans crushed tomatoes"",""1 (...",NaN,NaN,12,NaN,NaN,3,...,"{'healthy', 'dinner', 'low-calorie', 'low-carb'}",NaN,NaN,NaN,"bolognese ['crushed tomatoes', 'tomato sauce',...","['bolognese', '[', ""'crushed"", 'tomatoes', ""'""...","['bolognese', 'tomatoes', 'sauce', 'sausage', ...","['bolognese', 'tomato', 'sauce', 'sausage', 'r...","['bolognese', 'tomato', 'sauce', 'sausage', 'r...",bolognese tomato sauce sausage red pepper flak...
2,106878,bologna wedges,NaN,"[""8 slices of oscar meyer thin sliced bolo...",NaN,NaN,16,NaN,NaN,3,...,"{'appetizer', 'low-carb'}",NaN,NaN,NaN,"bologna wedges ['bologna', 'milk', 'cream chee...","['bologna', 'wedges', '[', ""'bologna"", ""'"", ',...","['bologna', 'wedges', 'cheese', 'cream', 'pick...","['bologna', 'wedge', 'cheese', 'cream', 'pickl...","['bologna', 'wedge', 'cheese', 'cream', 'pickl...",bologna wedge cheese cream pickle milk cream c...


In [14]:
def user_final_choice(options_user):
    print("Which one of these is the closest?")
    for index, row in options_user.iterrows():
        print(f"{index}: {row['recipe']}")
    # Error handling for user input
    while True:
        try:
            ind_user = int(input("Type the number corresponding to your choice: "))
            if ind_user < 0 or ind_user >= len(options_user):
                print("Invalid choice. Please enter a number within the range.")
            else:
                break
        except ValueError:
            print("Invalid input. Please enter a valid number.")

    # Select the chosen recipe on the original DF
    re_user = options_user.iloc[[ind_user]]
    return re_user

re_user = user_final_choice(options_user)


Which one of these is the closest?
0: lasagna
1: bolognese
2: bologna wedges
Invalid input. Please enter a valid number.
Invalid input. Please enter a valid number.


In [18]:
re_user["lems_string"][0]

'lasagna sausage paste sauce water noodle cheese garlic onion hot oil soft meat cook crumbly oil salt pepper spice tomato sauce tomato paste water uncovered minute shallow 9x13 inch pyrex dish put thin layer sauce half noodle sauce top half cheese layer ending mozzarella cheese degree minute stand minute serving square cottage cheese ricotta cheese mozzarella aunt finding recipe family fond cottage ricotta cheese recipe definitely favorite prefer ground beef italian sausage shredded mozzarella cheese instead sliced recipe call boiling water bother boiling'

In [19]:
### inputing the recipe

cv = CountVectorizer()
vege_df = dfs[(dfs["diet_id"] == 1) | (dfs["diet_id"] == 2)]
vege_df.reset_index(drop=True, inplace=True)

df_search = pd.concat([vege_df, re_user], axis=0, ignore_index=True)

df_search = df_search.reset_index(drop=True)
list_features = list(df_search["lems_string"])
count_matrix = cv.fit_transform(list_features)

cosine_sim = cosine_similarity(count_matrix)
similar_recipes = list(enumerate(cosine_sim[-1]))
sorted_similar_recipes = sorted(similar_recipes,key=lambda x:x[1],reverse=True)[1:]
suggested_ids = [d[0] for d in sorted_similar_recipes[:4]]
final_recipes = df_search.loc[suggested_ids]
final_recipes

,recipe_id,recipe,url,list_ingredients_raw,instructions_raw,category_id,serves,time_prep,nutrients_table,diet_id,...,search_terms,category,meal,meal_id,combined_features,tokens,tokens2,lemmatized,no_stop_word,lems_string
1571,106244,3 cheese spinach lasagna with tofu,NaN,"[""3 (16 ounce) jars spaghetti sauce"",""1 1/...",NaN,NaN,10,NaN,NaN,2,...,"{'italian', 'dinner', 'vegetarian', 'lunch'}",NaN,NaN,NaN,3 cheese spinach lasagna with tofu ['spaghetti...,"['3', 'cheese', 'spinach', 'lasagna', 'with', ...","['cheese', 'spinach', 'lasagna', 'with', 'tofu...","['cheese', 'spinach', 'lasagna', 'with', 'tofu...","['cheese', 'spinach', 'lasagna', 'tofu', 'sauc...",cheese spinach lasagna tofu sauce leaf oregano...
1229,103623,fettuccine with cream sauce for one,NaN,"[""2 ounces dry fettuccine pasta"",""2 tabl...",NaN,NaN,1,NaN,NaN,2,...,"{'diabetic', 'dinner', 'vegetarian'}",NaN,NaN,NaN,fettuccine with cream sauce for one ['fettucci...,"['fettuccine', 'with', 'cream', 'sauce', 'for'...","['fettuccine', 'with', 'cream', 'sauce', 'for'...","['fettuccine', 'with', 'cream', 'sauce', 'for'...","['fettuccine', 'cream', 'sauce', 'pasta', 'fat...",fettuccine cream sauce pasta fat cottage chees...
1161,102899,stinawina's lasagna rolls,NaN,"[""6 lasagna noodles, cooked "",""8 ounce...",NaN,NaN,6,NaN,NaN,2,...,"{'dinner', 'vegetarian', 'italian'}",NaN,NaN,NaN,"stinawina's lasagna rolls ['lasagna noodles', ...","['stinawina', ""'s"", 'lasagna', 'rolls', '[', ""...","['stinawina', 'lasagna', 'rolls', 'noodles', '...","['stinawina', 'lasagna', 'roll', 'noodle', 'ch...","['stinawina', 'lasagna', 'roll', 'noodle', 'ch...",stinawina lasagna roll noodle cheese cheese sa...
1051,101705,baked pesto pasta with tomato & cheese,NaN,"[""1 lb plain pasta or 1 lb whole whea...",NaN,NaN,6,NaN,NaN,2,...,"{'dinner', 'vegetarian', 'pasta', 'baked'}",NaN,NaN,NaN,baked pesto pasta with tomato & cheese ['pasta...,"['baked', 'pesto', 'pasta', 'with', 'tomato', ...","['baked', 'pesto', 'pasta', 'with', 'tomato', ...","['baked', 'pesto', 'pasta', 'with', 'tomato', ...","['baked', 'pesto', 'pasta', 'tomato', 'cheese'...",baked pesto pasta tomato cheese virgin olive o...


### what do they have in common

In [20]:
final_recipes["lems_string"]

1571    cheese spinach lasagna tofu sauce leaf oregano...
1229    fettuccine cream sauce pasta fat cottage chees...
1161    stinawina lasagna roll noodle cheese cheese sa...
1051    baked pesto pasta tomato cheese virgin olive o...
Name: lems_string, dtype: object

In [51]:
type(final_recipes["list_ingredients_raw"][1571 ])

str

In [25]:
final_recipes.index

Index([1571, 1229, 1161, 1051], dtype='int64')

In [27]:
recipe = final_recipes.loc[[1571]]
recipe

,recipe_id,recipe,url,list_ingredients_raw,instructions_raw,category_id,serves,time_prep,nutrients_table,diet_id,...,search_terms,category,meal,meal_id,combined_features,tokens,tokens2,lemmatized,no_stop_word,lems_string
1571,106244,3 cheese spinach lasagna with tofu,NaN,"[""3 (16 ounce) jars spaghetti sauce"",""1 1/...",NaN,NaN,10,NaN,NaN,2,...,"{'italian', 'dinner', 'vegetarian', 'lunch'}",NaN,NaN,NaN,3 cheese spinach lasagna with tofu ['spaghetti...,"['3', 'cheese', 'spinach', 'lasagna', 'with', ...","['cheese', 'spinach', 'lasagna', 'with', 'tofu...","['cheese', 'spinach', 'lasagna', 'with', 'tofu...","['cheese', 'spinach', 'lasagna', 'tofu', 'sauc...",cheese spinach lasagna tofu sauce leaf oregano...


In [65]:

for x in final_recipes.index:
    recipe = final_recipes.loc[x]
    name = recipe["recipe"].capitalize()
    #ingredients = ','.join(recipe["list_instructions"])
    ingredients = recipe["list_ingredients_raw"].replace("', '"," " ).replace("['", "").replace("']", "").replace("nan ", "").capitalize()
    steps = recipe["list_instructions"].replace("', '"," " ).replace("['", "").replace("']", "").replace("nan ", "").capitalize()

    #if name and ingredients and steps: 
    print(f"RECIPE: {name}")
    print(f"INGREDIENTS: {ingredients}")
    print(f"STEPS: {steps}")
    print("\n")

RECIPE: 3 cheese spinach lasagna with tofu
INGREDIENTS: ["3 (16   ounce) jars   spaghetti sauce","1 1/2  cups    water","2       bay leaves","1   tablespoon    dried oregano leaves, crushed ","1/2  tablespoon    ground marjoram","1   teaspoon    salt","1   teaspoon    pepper","1   lb    tofu, drained \u0026 crumbled ","4   cups    mozzarella cheese, shredded ","32   ounces    cottage cheese or 32   ounces    ricotta cheese","1/4  cup    dried parsley","2   lbs    fresh spinach, chopped ","1   lb    lasagna noodle (uncooked)","1/4  cup    parmesan cheese, grated "]
STEPS: Preheat oven to 350°f. bring spaghetti sauce and water to a boil; add next 5 seasonings if desired, and reduce heat to a simmer (sauce should still be a little runny when lasagna is assembled; the uncooked noodles will soak up the extra moisture). while sauce is simmering, combine tofu, mozzarella, cottage cheese and parsely in a large mixing bowl and stir. assemble the lasagna as follows: (1) spray the bottom of two 1

In [67]:

for x in final_recipes.index:
    recipe = final_recipes.loc[x]
    name = recipe["recipe"].capitalize()
    lem_string = recipe["lems_string"].capitalize()
    

    #ingredients = ','.join(recipe["list_instructions"])
    #ingredients = recipe["list_ingredients_raw"].replace("', '"," " ).replace("['", "").replace("']", "").replace("nan ", "").capitalize()
    #steps = recipe["list_instructions"].replace("', '"," " ).replace("['", "").replace("']", "").replace("nan ", "").capitalize()

    #if name and ingredients and steps: 
    print(f"RECIPE: {name}")
    print(f"STRING: {lem_string}")
    #print(f"STEPS: {steps}")
    print("\n")

RECIPE: 3 cheese spinach lasagna with tofu
STRING: Cheese spinach lasagna tofu sauce leaf oregano leaf marjoram cheese cheese parsley spinach noodle cheese oven spaghetti sauce water boil add seasoning desired reduce heat simmer sauce runny lasagna assembled uncooked noodle soak extra moisture sauce simmering combine tofu mozzarella cottage cheese parsely large mixing bowl stir lasagna spray bottom x9 x2 casserole pan pam spoon sauce cover bottom place layer noodle top sauce spread cheese mixture evenly noodle pan sprinkle spinach cheese sauce evenly top layer ending sauce top parmesan cheese tightly aluminum foil bake 350f hour healthier version mozzarella cottage cheese freeze pan put freezer cooking cook hour instead hour ready wa modified recipe real cookin low fat


RECIPE: Fettuccine with cream sauce for one
STRING: Fettuccine cream sauce pasta fat cottage cheese ricotta cheese buttermilk cheese fettucine boiling water uncovered minute cheese buttermilk blender food processor smo

### Final Fonction:

In [ ]:
#cv_char = CountVectorizer(analyzer='char_wb', ngram_range = (2,2))
list_recipes = list(dfs["recipe"])

In [ ]:
#cv = CountVectorizer()

In [ ]:
def user_recipe():
    '''gives the user an input , returns the result'''
    user_recipe = input("Give me a recipe that you like:")
    return user_recipe

In [ ]:
def search_recipe_name(user_recipe, list_recipes):
    '''takes the result of the input , as well as the list of recipes of the database
    user returns a df with the row containg the user recipe or the closest to it'''
    list_recipes = list_recipes.append(user_recipe)
    count_matrix = cv_char.fit_transform(list_recipes)
    cosine_sim = cosine_similarity(count_matrix)
    similar_recipes = list(enumerate(cosine_sim[-1]))
    sorted_similar_recipes = sorted(similar_recipes,key=lambda x:x[1],reverse=True)[1:]
    suggested_id = [d[0] for d in sorted_similar_recipes[:1]]
    re_user = dfs.loc[suggested_id]
    return re_user

In [ ]:
def search_final_recipe(vege_df, re_user):
    cv = CountVectorizer()
    df_search = pd.concat(vege_df, re_user, axis=0)
    df_search = df_search.reset_index(drop=True)
    list_features = list(df_search["lemmatized"])
    

In [ ]:
def print_recipe(suggested_ids):
    print(f"Top similar recipes to {user_recipe}")
    for x in suggested_ids:
        final_recipes.loc[x]
        name = final_recipes["recipe"]
        ingredients = final_recipes["list_ingredients"]
        steps = final_recipes["list_instructions"]
        if name and ingredients and steps: 
            print(f"Recipe: {name}")
            print(f"Ingredients: {ingredients}")
            print(f"Steps: {steps}")
        